In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler

import os
import re

Using TensorFlow backend.


In [2]:
EMBEDDING_FILES = [
    'embeddings/crawl-300d-2M.vec',
    'embeddings/glove.840B.300d.txt'
]

NUM_MODELS = 2
BATCH_SIZE = 128
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 220

# Helper Functiosn

## Processing Embeddings

In [3]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)


def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

## Build and define the model

In [4]:
def build_model(embedding_matrix, model_name):
    words = Input(shape=(max_length,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='tanh')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    model.name = model_name

    return model

# Data Preprocessing

In [5]:
def load_directory_data(directory):
    data={}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

def load_dataset(directory):
    pos_df = load_directory_data(directory + "/pos")
    neg_df = load_directory_data(directory + "/neg")
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

In [6]:
train_df = load_dataset("data/aclImdb/train/")
test_df = load_dataset("data/aclImdb/test/")

In [7]:
x_train = train_df["sentence"]
y_train = train_df["polarity"]

x_test = test_df["sentence"]
y_test = test_df["polarity"]

In [8]:
# from keras.utils import to_categorical

# y_train = to_categorical(train_df["sentiment"])
# y_test = to_categorical(test_df["sentiment"])

In [9]:
set(train_df['sentiment'].values), train_df["sentiment"][1], y_train[1]

({'1', '10', '2', '3', '4', '7', '8', '9'}, '1', 0)

In [10]:
x_train[1], y_train[1]

('I have never seen a comedy that was this much of a chore to sit thru...not one laugh in it. Ok, maybe one little chuckle for the Michael Clarke Duncan bit as the big, black, bald gay virgin. But the rest of it was shockingly un-funny. On top of being void of any laughs the "skits" go on forever! Steer clear of this one if you value your time and money. DREADFUL!!! The worst!!!',
 0)

## Tokenisation

In [11]:
tokenizer_obj = text.Tokenizer()
total_reviews = x_train + x_test
tokenizer_obj.fit_on_texts(total_reviews)

max_length = max([len(s.split()) for s in total_reviews])
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer_obj.texts_to_sequences(x_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(x_test)

X_train_pad = sequence.pad_sequences(X_train_tokens, maxlen=max_length, padding="post")
X_test_pad = sequence.pad_sequences(X_test_tokens, maxlen=max_length, padding="post")

In [12]:
# embedding_matrix = np.concatenate(
#     [build_matrix(tokenizer_obj.word_index, f) for f in EMBEDDING_FILES], axis=-1)

# np.savetxt('embedding_concat.txt', embedding_matrix , fmt='%d')

In [13]:
embedding_matrix = np.loadtxt('embedding_concat.txt', dtype=int)

# Training Loop

In [14]:

from keras.callbacks import Callback
import keras.backend as K
import numpy as np

class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

In [15]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, TensorBoard
LOG_DIR = '/media/eigenstir/1TBSecondary/tbgraphs'

tbCallBack = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7, min_delta=0.0001)
mc = ModelCheckpoint('best_model_cudnnlstm.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
ls_sched = SGDRScheduler(1e-2, 1e-1, np.ceil(25000/BATCH_SIZE))
# ls_sched = LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** epoch))

In [16]:
model = build_model(embedding_matrix, model_name="cudnnlstm")
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3173)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 3173, 600)    75367800    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 3173, 600)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 3173, 256)    747520      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
bidirectio

In [17]:
model.fit(X_train_pad,
            y_train,
            validation_data=(X_test_pad, y_test),
            batch_size=BATCH_SIZE,
            epochs=20,
            callbacks=[tbCallBack, es, mc]
        )

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 234s 9ms/step - loss: 0.6827 - acc: 0.5540 - val_loss: 0.6527 - val_acc: 0.6247

Epoch 00001: val_acc improved from -inf to 0.62472, saving model to best_model_cudnnlstm.h5
Epoch 2/20
25000/25000 [==============================] - 244s 10ms/step - loss: 0.6298 - acc: 0.6430 - val_loss: 0.5850 - val_acc: 0.6926

Epoch 00002: val_acc improved from 0.62472 to 0.69256, saving model to best_model_cudnnlstm.h5
Epoch 3/20
25000/25000 [==============================] - 244s 10ms/step - loss: 0.5883 - acc: 0.6840 - val_loss: 0.5823 - val_acc: 0.6786

Epoch 00003: val_acc did not improve from 0.69256
Epoch 4/20
25000/25000 [==============================] - 246s 10ms/step - loss: 0.5577 - acc: 0.7094 - val_loss: 0.5420 - val_acc: 0.7268

Epoch 00004: val_acc improved from 0.69256 to 0.72680, saving model to best_model_cudnnlstm.h5
Epoch 5/20
25000/25000 [==============================] - 2

# Test Time

In [30]:
model = build_model(embedding_matrix, 'test')
model.load_weights('best_model_cudnnlstm.h5')

In [46]:
idx = 2
test_text = test_df["sentence"][idx]
test_df["sentence"][idx], test_df["polarity"][idx]

("When I saw this movie i expected it to be a cheesy American movie done on the cheap with appalling actors. I was really surprised to find that i was totally wrong. The movie centres around Bartely or B who has been rejected from all of his colleges- the actor who play B is very natural and makes his character seem real- and decides to create a pretend school so his parents stop harassing him. However loads of people see his fake website and join. Feeling their sorrows B can't turn them away much to the chagrin of his best mate. The college is the ideal place with you learning what you want or doing nothing. The school faces opposition from the proper college which ends up closing it down. The film ends on a high and i recommend you watching it. Its does have it flaws but it is a feel good cheerful film with a few unpredictable twists.",
 1)

In [47]:
test_text = tokenizer_obj.texts_to_sequences(test_text)
test_text = sequence.pad_sequences(test_text, maxlen=max_length, padding="post")

In [48]:
prediction = model.predict(test_text)

In [49]:
prediction[0]

array([0.62270343], dtype=float32)

Testing texts: Uber

Neg:

1. #uber's customer service department is worst than comcast.
1. #Uber ...get your act right! # frustrated#waiting for competition
1. .@Uber charges more if they think you're willing to pay more. Uber thinks you're willing to pay more when your battery is about to die. And Uber knows when your battery is about to die.

Pos:
1. Ride-hailing industry expected to grow eightfold to $285 billion by 2030
1. Uber is awesome!
1. Best service ever with Uber


In [139]:
imdb = tf.keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 3s 0us/step


# Business Times Singapore

In [1]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

ModuleNotFoundError: No module named 'bs4'

# Backtrader


In [ ]:
import backtrader as bt
import backtrader.indicators as btind
import datetime
import os.path
import sys

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.15,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])
    
    def next(self):
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment


class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)     
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    ### Main Strat ###
    def next(self):
        # log closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        
        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.sentiment)
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)
        

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    # Strategy
    cerebro.addstrategy(SentimentStrat)

    # Data Feed
    data = bt.feeds.YahooFinanceData(
        dataname = 'FB',
        fromdate = earliest_date,
        todate = datetime.datetime(2018,11,25),
        reverse = False
    )
    
    cerebro.adddata(data)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

# Saving a model

In [ ]:
import sys
from keras.models import load_model
import tensorflow as tf
# from keras import backend as K
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants



In [ ]:
keras.backend.set_learning_phase(0)

model = build_model(embedding_matrix, model_name = str(model_idx))
config = model.get_config()
weights = model.get_weights()
new_model = Model.from_config(config)
new_model.set_weights(weights)

Export the model

In [ ]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

export_path = 'exported_model'
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'text': new_model.input},
                                  outputs={'sentiment': new_model.output})

with keras.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

# Another type of loop

In [15]:
embedding_matrix = np.loadtxt('embedding_concat.txt', dtype=int)
words = Input(shape=(max_length,))
x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
x = SpatialDropout1D(0.3)(x)
x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
result = Dense(11, activation='sigmoid')(hidden)
    
model = Model(inputs=words, outputs=result)
model.compile(loss='binary_crossentropy', optimizer='adam')

IndentationError: unexpected indent (<ipython-input-15-c5479d416f9e>, line 3)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

tbCallBack = keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
ls_sched = LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))

In [ ]:
model.fit(X_train_pad, y_train, batch_size=64, epochs=10, 
          validation_data=(X_test_pad, y_test), callbacks=[tbCallBack, es, mc])